# Simplified Shell Hackathon problem

How many parking slots with chargers installed as in 2018 do we need to open to fulfill electricity demand of 2018?

This is a capacitated FLP completely analogeous to our example 2, except parking slot/facility capacities vary.

In [1]:
from math import sqrt
from itertools import product

import pandas as pd

from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit

In [2]:
supply = pd.read_csv("../data/raw/exisiting_EV_infrastructure_2018.csv")
demand = pd.read_csv("../data/raw/Demand_History.csv")

In [3]:
slow_charger = 200
fast_charger = 400

In [4]:
setup_cost = 1
# assume fixed capacities of parking slots 
capacities = (supply["existing_num_SCS"]*slow_charger + supply["existing_num_FCS"]*fast_charger).tolist()
demands = demand["2018"].tolist()

In [5]:
sum(capacities) > sum(demands), sum(capacities), sum(demands)

(True, 361600, 361529.6365968905)

The total capacity is suspiciously close to total demand. Are the numbers of shell hackathon problem generated artificially? This suggest that we definitely need all parking slots opened.

In [6]:
# facilities are parking slots
facilities = list(supply[["x_coordinate","y_coordinate"]].itertuples(index=False, name=None))
# customers are areas of electricity demand
customers = list(demand[["x_coordinate","y_coordinate"]].itertuples(index=False, name=None))

In [7]:
# Compute key parameters of MIP model formulation
num_facilities = len(facilities)
num_customers = len(customers)
cartesian_prod = list(product(range(num_customers), range(num_facilities)))

In [8]:
# This function determines the Euclidean distance between a facility and customer sites.
def compute_distance(loc1, loc2):
    dx = loc1[0] - loc2[0]
    dy = loc1[1] - loc2[1]
    return sqrt(dx*dx + dy*dy)

In [9]:
# Compute distance matrix
distance = {(c,f): compute_distance(customers[c], facilities[f]) for c, f in cartesian_prod}

In [10]:
solver = pywraplp.Solver.CreateSolver('SCIP_MIXED_INTEGER_PROGRAMMING')

In [11]:
# Variables
select = {}
for j in range(num_facilities):
    select[j] = solver.BoolVar("Select")
assign = {}
for i,j in distance.keys(): 
    assign[(i,j)] = solver.NumVar(0,solver.infinity(),"Assign")    

In [12]:
# demand constraints
for i in range(num_customers):
    solver.Add(sum(assign[(i,j)] for j in range(num_facilities)) == demands[i] )
# shipping constraints
for i,j in distance.keys():
    solver.Add(assign[(i,j)] <= select[j]*demands[i])
# capacity constraints
for j in range(num_facilities):
    solver.Add(sum(assign[(i,j)] for i in range(num_customers)) <= select[j]*capacities[j])    

In [13]:
objective = solver.Objective()
# parking slot opening costs
for j in range(num_facilities):
    objective.SetCoefficient(select[j], setup_cost)
# distance costs       
for i in range(num_customers):
    for j in range(num_facilities):
        objective.SetCoefficient(assign[(i,j)],distance[(i,j)])
objective.SetMinimization()        

In [14]:
status = solver.Solve()

In [15]:
if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', solver.Objective().Value())
    print()
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
else:
    print('The problem does not have an optimal solution.')

Objective value = 2101966.1927448846

Problem solved in 16493.000000 milliseconds
Problem solved in 19474 iterations
Problem solved in 1 branch-and-bound nodes


In [16]:
# display optimal values of decision variables
lines = []
for facility in select.keys():
    if (abs(select[facility].solution_value()) > 1e-6):
        lines.append(f"Open a parking lot at location {facility + 1}.")
for line in lines[:5]: print(line)
print("...")
for line in lines[-5:]: print(line)
    
print("Number of parking slots open:", len(lines))    

Open a parking lot at location 1.
Open a parking lot at location 2.
Open a parking lot at location 3.
Open a parking lot at location 4.
Open a parking lot at location 5.
...
Open a parking lot at location 96.
Open a parking lot at location 97.
Open a parking lot at location 98.
Open a parking lot at location 99.
Open a parking lot at location 100.
Number of parking slots open: 100


In [17]:
# Shipments from facilities to customers.
lines = []
for customer, facility in assign.keys():
    if (abs(assign[customer, facility].solution_value()) > 1e-6):
        lines.append(f"Demand point {customer + 1} receives {round(assign[customer, facility].solution_value(), 2)} of its demand {demands[customer]} from parking slot {facility + 1} .")
for line in lines[:5]: print(line)
print("...")
for line in lines[-5:]: print(line)

Demand point 1 receives 13.12 of its demand 13.119572063684156 from parking slot 39 .
Demand point 2 receives 12.02 of its demand 12.020090866258569 from parking slot 39 .
Demand point 3 receives 14.02 of its demand 14.018253749831027 from parking slot 39 .
Demand point 4 receives 15.01 of its demand 15.012302116613936 from parking slot 39 .
Demand point 5 receives 16.36 of its demand 16.355563054111933 from parking slot 39 .
...
Demand point 4092 receives 5.43 of its demand 5.426192736874994 from parking slot 3 .
Demand point 4093 receives 2.06 of its demand 2.057449824611969 from parking slot 3 .
Demand point 4094 receives 3.22 of its demand 3.218518575941802 from parking slot 3 .
Demand point 4095 receives 6.26 of its demand 6.262573820751901 from parking slot 3 .
Demand point 4096 receives 6.86 of its demand 6.860939370983753 from parking slot 3 .
